<a href="https://colab.research.google.com/github/JohnEaganFS/CSCI191T/blob/main/Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CSCI 191T - Machine Learning
##Name: John Eagan 
##Assignment: Backpropagation

In [ ]:
import math
import numpy as np
import random
import plotly.express as px
import sklearn
from sklearn.metrics import accuracy_score
import pandas as pd

## F(w, x), Sigmoid, Classify, and Predict Functions

In [ ]:
def F(W,X):
  return sum([w*x for w,x in zip(W,X)])

In [ ]:
def sigmoid(x):
  return 1.0/(1+math.exp(-x))

In [ ]:
def relu(x):
  return max([0.0, x])

In [ ]:
def classify(model, X):
  return [1 if sigmoid(F(model, x)) > 0.5 else 0 for x in X]

In [ ]:
def predict(model, x):
  w1 = model[0]
  w2 = model[1]
  v1 = model[2]
  z1 = sigmoid(F(w1, x))
  z2 = sigmoid(F(w2, x))
  Z = (1, z1, z2)
  y = sigmoid(F(v1,Z))
  return y

In [ ]:
def classifyY(Y):
  return [1 if y > 0.5 else 0 for y in Y]

# Training Data

In [ ]:
data = [ ( (0,0), 0 ), ( (0,1), 1 ), ( (1,0), 1), ( (1,1), 0 ) ]

#Randomizing Weights

In [ ]:
np.random.seed(42)
def randomWeights(weightVectors, range):
  newWeightVectors = []
  for weightVector in weightVectors:
    newWeightVectors.append([np.random.uniform(-1*range, range) for w in weightVector])
  return newWeightVectors

In [ ]:
w1 = [0, 0, 0]
w2 = [0, 0, 0]
v1 = [0, 0, 0]
[w1, w2, v1] = randomWeights([w1, w2, v1], 1)
model = [w1, w2, v1]
print(model)

[[-0.9152421199071252, -0.20564741686637378, 0.5269973699737718], [0.3271072350434838, -0.2565094314910705, -0.8951948670463146], [-0.41917545299916914, -0.6743595638997375, -0.4685845685401149]]


# Initial Training and Test

In [ ]:
alpha = 0.3

In [ ]:
trainingData = data.copy()
trainingData = [((1, x1, x2), r) for ((x1, x2), r) in trainingData]
print(trainingData)

[((1, 0, 0), 0), ((1, 0, 1), 1), ((1, 1, 0), 1), ((1, 1, 1), 0)]


In [ ]:
def Backpropagation(model, learningRate, data, iterations, SEAD):
  w1 = model[0]
  w2 = model[1]
  v1 = model[2]
  for i in range(iterations):
    np.random.shuffle(data)
    
    # Tracking Squared Error and Accuracy (every 100 epochs)
    if (i % 100) == 0:
      testData = [ ( (0,0), 0 ), ( (0,1), 1 ), ( (1,0), 1), ( (1,1), 0 ) ]
      X = [(1, x1, x2) for ((x1, x2),r) in testData]
      R = [r for ((x1, x2),r) in testData]
      yPred = [predict([w1, w2, v1], x) for x in X]
      squaredError = sum([pow(r - y, 2) for (r, y) in zip(R, yPred)])

      classPred = classifyY(yPred)
      accuracy = sklearn.metrics.accuracy_score(R, classPred)

      SEAD.append((squaredError, accuracy))

    # Backpropagation
    for (x, r) in data:
      # Forward Pass
      z1 = sigmoid(F(w1, x))
      z2 = sigmoid(F(w2, x))
      Z = (1, z1, z2)
      y = sigmoid(F(v1,Z))

      # Backward Pass
      deltaV = [learningRate * (r - y) * z for z in Z]
      deltaW1 = [learningRate * (r - y) * v1[1] * z1 * (1 - z1) * xi for xi in x]
      deltaW2 = [learningRate * (r - y) * v1[2] * z2 * (1 - z2) * xi for xi in x]

      v1 = [v + dv for (v, dv) in zip(v1, deltaV)]
      w1 = [w + dw for (w, dw) in zip(w1, deltaW1)]
      w2 = [w + dw for (w, dw) in zip(w2, deltaW2)]

  newModel = [w1, w2, v1]
  return newModel

In [ ]:
squaredErrorAccuracyData = []
model = Backpropagation(model, alpha, trainingData, 1000, squaredErrorAccuracyData)
print("Final Model:", model)

Final Model: [[-3.2591063578772608, 7.2501404610282085, 7.250693733234696], [7.844258083236938, -5.242393717995175, -5.240495698155427], [-12.611966932095935, 8.538539302252513, 8.696696157567322]]


In [ ]:
X = [(1,x1,x2) for ((x1,x2),r) in data]
print("Test Data:", X)
yPred = [predict(model, x) for x in X]
print("Y Predictions:", yPred)
print("Class Predictions:", classifyY(yPred))

Test Data: [(1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
Y Predictions: [0.026526052706117426, 0.979552701087097, 0.9795297770962561, 0.02949690327923357]
Class Predictions: [0, 1, 1, 0]


# Testing 100 Random Models

The inital random weights seem to have a substantial effect on the final model because of local minima in the XOR error surface.

In [ ]:
successCount = 0
failureCount = 0
for i in range(100):
  [w1, w2, v1] = randomWeights([w1, w2, v1], 1)
  model = [w1, w2, v1]
  SEAD = []
  model = Backpropagation(model, alpha, trainingData, 1000, SEAD)
  yPred = [predict(model, x) for x in X]
  if (classifyY(yPred) == [0,1,1,0]):
    successCount += 1
  else:
    failureCount += 1
print(successCount)
print(failureCount)

68
32


Out of 100 models, about 2/3 were able to model XOR.

# 5 Different Initial Weights

In [ ]:
modelList = [randomWeights([[0,0,0],[0,0,0],[0,0,0]], 1) for i in range(5)]

## Initial Random Weights

In [ ]:
for i, model in enumerate(modelList):
  print("Model", str(i) + ":", "\n", np.array(model), "\n")

# Model i:
# w1
# w2
# v1

Model 0: 
 [[ 0.40144497  0.09380559  0.80415034]
 [-0.29678894  0.96171604 -0.07489762]
 [-0.38776761  0.26254586 -0.46330728]] 

Model 1: 
 [[ 0.09194924  0.19342716  0.3546975 ]
 [-0.58023909  0.70011304 -0.86897872]
 [ 0.40417753  0.21851434  0.71328065]] 

Model 2: 
 [[ 0.7398408   0.65043978 -0.03921999]
 [ 0.64550828  0.72055007  0.48186552]
 [ 0.68163987 -0.90306136 -0.04615828]] 

Model 3: 
 [[ 0.48955145  0.92532185 -0.42668866]
 [ 0.54465499  0.5206087   0.08426143]
 [ 0.41788408  0.94544924 -0.30084753]] 

Model 4: 
 [[-0.03546604 -0.43428262  0.36246929]
 [ 0.39096286 -0.85554524  0.69462158]
 [ 0.74081185  0.94871367 -0.73903522]] 



## Learning Rate

In [ ]:
alpha = 0.3

## Final Weights

In [ ]:
trainingData = [((1, x1, x2), r) for ((x1, x2), r) in data]
print(trainingData)

[((1, 0, 0), 0), ((1, 0, 1), 1), ((1, 1, 0), 1), ((1, 1, 1), 0)]


In [ ]:
SEADList = [[] for i in range(len(modelList))]
for i, model in enumerate(modelList):
  modelList[i] = Backpropagation(model, alpha, trainingData, 1000, SEADList[i])
  trainingData = [((1, x1, x2), r) for ((x1, x2), r) in data] # Resetting data to default order (not necessary)

for i, model in enumerate(modelList):
  print("Final Model", str(i) + ":", "\n", np.array(model), "\n")

Final Model 0: 
 [[-3.07424817 -5.86920859  5.57045922]
 [-3.35989116  6.02803863 -6.22743746]
 [-4.45990685  9.15540143  9.09453059]] 

Final Model 1: 
 [[ 2.84872635  6.13189498 -5.80092658]
 [-3.68718194  6.76234916 -6.81517252]
 [ 4.5450236  -9.7344899  10.18019972]] 

Final Model 2: 
 [[-1.3612497   2.34607257 -5.35994298]
 [ 1.09432886 -4.69144958 -6.07153633]
 [ 0.03074398  4.32983696 -5.9585092 ]] 

Final Model 3: 
 [[-3.63065011e+00  5.10789942e+00 -7.76534774e+00]
 [ 1.03850287e+00 -2.94330713e+00 -6.86131290e+00]
 [-1.56086023e-03  6.95787527e+00 -6.07663613e+00]] 

Final Model 4: 
 [[ -3.2546731   -6.33713978   6.08812627]
 [  3.16726574  -6.32315429   6.55436042]
 [  4.72891391  10.72110284 -10.15113389]] 



## Outputs for Each Model

In [ ]:
print(X)
for i, model in enumerate(modelList):
  yPred = [predict(model, x) for x in X]
  print("Model", i, "Output:", "\n", yPred, "\n", classifyY(yPred), "\n")

[(1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
Model 0 Output: 
 [0.022975714621255642, 0.9819991329539177, 0.982804671633787, 0.019748243030223222] 
 [0, 1, 1, 0] 

Model 1 Output: 
 [0.012035082871906556, 0.9830793482345941, 0.9894661393222931, 0.010307828468698104] 
 [0, 1, 1, 0] 

Model 2 Output: 
 [0.027925310924340165, 0.4987907705046413, 0.9536563782852551, 0.5210345408251951] 
 [0, 0, 1, 1] 

Model 3 Output: 
 [0.01325664865151599, 0.4951471403310247, 0.9924296487128177, 0.5025993615644921] 
 [0, 0, 1, 1] 

Model 4 Output: 
 [0.009819087362259766, 0.9910224454838868, 0.986808090389387, 0.008319799958354749] 
 [0, 1, 1, 0] 



## Squared Error and Accuracy

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
SE = [[se for (se, a) in sead] for sead in SEADList]
Acc = [[a for (se, a) in sead] for sead in SEADList]

columns = []
columns = columns + ['Epoch ' + str(i * 100) for i in range(10)]
SE = pd.DataFrame(SE, columns=columns)
SE.index.names = ['Model']
print("Squared Error")
SE

Squared Error


,Epoch 0,Epoch 100,Epoch 200,Epoch 300,Epoch 400,Epoch 500,Epoch 600,Epoch 700,Epoch 800,Epoch 900
Model,,,,,,,,,,
0,1.049531,1.001144,0.999445,0.997848,0.985119,0.802010,0.106132,0.015508,0.005371,0.002628
1,1.134406,0.944655,0.752107,0.621130,0.067221,0.011670,0.004408,0.002264,0.001366,0.000910
2,1.000806,0.999765,1.000047,1.001012,0.999910,1.000202,0.992898,0.928489,0.662448,0.550678
3,1.165208,0.998426,0.989714,0.869044,0.645818,0.550653,0.525296,0.516421,0.511938,0.509461
4,1.133342,0.927051,0.148535,0.018930,0.006148,0.002915,0.001673,0.001079,0.000750,0.000550


In [ ]:
Acc = pd.DataFrame(Acc, columns=columns)
Acc.index.names = ['Model']
print("Accuracy")
Acc

Accuracy


,Epoch 0,Epoch 100,Epoch 200,Epoch 300,Epoch 400,Epoch 500,Epoch 600,Epoch 700,Epoch 800,Epoch 900
Model,,,,,,,,,,
0,0.5,0.50,0.50,0.25,0.50,0.75,1.0,1.0,1.00,1.00
1,0.5,0.50,0.75,0.75,1.00,1.00,1.0,1.0,1.00,1.00
2,0.5,0.75,0.75,0.50,0.50,0.50,0.5,0.5,0.75,0.50
3,0.5,0.25,0.50,0.75,0.75,0.75,0.5,0.5,0.50,0.75
4,0.5,0.75,1.00,1.00,1.00,1.00,1.0,1.0,1.00,1.00
